In [1]:
import os
import json
import zipfile
import requests
from urllib.request import urlretrieve

import pandas as pd
import re
import glob
import time


## Download Data

In [2]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}/versions/3"
headers = {"Content-Type": "application/json"}


response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)["files"]

download_url = [item["download_url"] for item in data if item["name"] == "data.zip"][0]
download_url

output_directory = "data/"
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
    
urlretrieve(download_url, output_directory + "rainfall.zip")

with zipfile.ZipFile(os.path.join(output_directory, "rainfall.zip"), 'r') as f:
    f.extractall(output_directory)
# output_directory = "figshareairline/"

## Combine CSV

### Combine all the CSVs using Python

In [4]:
%%time
files = glob.glob("data/*.csv")
files.remove("data/observed_daily_rainfall_SYD.csv")
df = pd.concat(
    (pd.read_csv(file, index_col=0, parse_dates=['time'])
     .assign(model=re.findall(r'[^\/&\\]+(?=_daily_rainfall_NSW\.)', file)[0])
        )
     for file in files
)
df.to_csv("data/combined_data.csv")


CPU times: user 6min 42s, sys: 11.3 s, total: 6min 53s
Wall time: 7min 18s


In [5]:
df.head(5)

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


#### Combine CSVs Time taken in each machine



## Observations